## Read data from database

In [1]:
import os
# specify which GPU will be used
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import os

import pymysql
from warnings import filterwarnings

_connection = None

def get_connection(db_config):
    """
    get db connection
    :return:
    """
    global _connection
    if _connection is None:
        _connection = pymysql.connect(host=db_config['host'], user=db_config['username'],
                                      password=db_config['password'],
                                      db=db_config['db'], charset="utf8")
        filterwarnings('ignore', category=pymysql.Warning)

    return _connection


def close():
    """
    close DB connection
    :return:
    """
    global _connection
    if _connection is not None:
        _connection.close()
    _connection = None

In [3]:
db = {
    'host': '172.26.187.242',
    'username': 'malware_r',
    'password': 'GEg22v2O7jbfWhb3',
    'db': 'malware'
}

### Fields

- mw_file_suffix: file name after hash value
- mw_file_prefix: directory
- mw_em_f: features of ember, splitted by ";"

In [4]:
import time

# the base function which can query sql and return dict data
def get_specific_data(table_suffix, sql=None):
    start_time = time.time()
    
    global _connection
    if _connection is None:
        raise Exception("please init db connect first")

    cursor = _connection.cursor()
    cursor.execute("SET NAMES utf8mb4")

    ret = []
    
    sql = """
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  right join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  right join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where a.section_name = '.text' and c.mw_num_engines <> -1 and (c.mw_num_engines > 6 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201701', '201703')
group by mw_file_hash
    """ % (table_suffix, table_suffix, table_suffix)
    
    if sql is None:
        sql = sql
    cursor.execute(sql)

    field_names = [i[0] for i in cursor.description]

    for row in cursor:
        temp = {}
        for key in range(len(row)):
            temp[field_names[key]] = row[key]
        ret.append(temp)
     
    cursor.close()
    # _connection.close()
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return ret

In [5]:
close()
res = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    res.extend(get_specific_data(suffix))
close()
print(len(res))

--- 10.697633743286133 seconds ---
--- 7.454339027404785 seconds ---
--- 8.65286898612976 seconds ---
--- 6.7891435623168945 seconds ---
--- 6.066123008728027 seconds ---
--- 5.753019332885742 seconds ---
--- 5.706942081451416 seconds ---
--- 5.959042310714722 seconds ---
--- 5.935366868972778 seconds ---
--- 7.311773777008057 seconds ---
--- 5.53498387336731 seconds ---
--- 5.935747861862183 seconds ---
--- 5.093973159790039 seconds ---
--- 4.8209123611450195 seconds ---
--- 4.777883768081665 seconds ---
--- 4.56196928024292 seconds ---
239085


## Check and split data

In [6]:
import pandas as pd
data = pd.DataFrame(res)
del res

In [7]:
data.head()

,mw_em_f,mw_file_directory,mw_file_hash,mw_file_size,mw_num_engines,pointerto_raw_data,section_name,virtual_size
0,0.21537522971630094;0.02217341773211956;0.0131...,201703,000002D8A582B2083F1F2DEF365A82234766D546EC8021...,_958490,0,1024,.text,440152
1,0.2265700697898865;0.01793677918612957;0.01251...,201703,000043B9D37086041959E44EB6EED77F05DA44492F3B6A...,_6343168,0,634368,.text,5549270
2,0.30814531445503235;0.004307170398533343;0.003...,201701,0000A3EAB7F06DB29113F7BDE61328D56BC874B16DD895...,_339434,11,1024,.text,13352
3,0.7433035969734192;0.004743303637951613;0.0041...,201701,0000D95594F0BC57CA94FD90C9477A7B984A8429B7A45B...,_3584,7,1024,.text,566
4,0.013131157495081423;0.004108430817723274;0.00...,201701,00015BCA398B78199C5442DB5B28D14405549FB135FA69...,_2098368,0,1024,.text,25467


In [8]:
import numpy as np
import scipy.stats as stats
import pylab as pl

max_length = 300000

# only keep the data whoose length less than max_length
clean_data = data.loc[data.virtual_size <= max_length]
clean_data = clean_data.reset_index(drop=True)
del data

print(clean_data.shape)

h = sorted(clean_data.virtual_size.ravel())  #sorted

fit = stats.norm.pdf(h, np.mean(h), np.std(h))  #this is a fitting indeed

pl.plot(h,fit,'-o')

pl.hist(h,normed=True)      #use this to draw histogram of your data

pl.show()                   #use may also need add this 

(195320, 8)


/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


<Figure size 640x480 with 1 Axes>

In [9]:
# re-value label
clean_data.mw_num_engines[clean_data.mw_num_engines == 0 ] = 0
clean_data.mw_num_engines[clean_data.mw_num_engines > 6 ] = 1
label = clean_data.mw_num_engines.ravel()
label

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


array([1, 1, 0, ..., 1, 0, 1])

In [10]:
from sklearn.model_selection import train_test_split

x_tmp, x_test, y_tmp, y_test = train_test_split(clean_data, label, test_size=0.1)
x_train, x_val, y_train, y_val = train_test_split(x_tmp, y_tmp, test_size=0.1)
del x_tmp, y_tmp

In [11]:
x_train = x_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)

### How to get the binary data

In [12]:
import lief

base_path = "/ssd/2017/{0}/{1}{2}" # at ncrs or ncrd
item = x_train.iloc[0]
file_path = base_path.format(item["mw_file_directory"], item["mw_file_hash"], item["mw_file_size"])
lief_binary = lief.PE.parse(file_path)
print(lief_binary.sections[0])

.text     3428      1000      3600      400       0         6.56354   CNT_CODE - MEM_EXECUTE - MEM_READ - MEM_WRITE


### How to get the section's bytes

In [13]:
in_file = open(file_path, 'rb')
in_file.seek(item['pointerto_raw_data'])
bytes_data = [int(single_byte) for single_byte in in_file.read(item['virtual_size'])]
bytes_data

[124,
 60,
 156,
 123,
 134,
 70,
 135,
 61,
 65,
 193,
 209,
 227,
 151,
 206,
 88,
 245,
 166,
 158,
 153,
 232,
 126,
 184,
 105,
 159,
 2,
 208,
 23,
 212,
 240,
 252,
 65,
 241,
 0,
 108,
 136,
 19,
 31,
 82,
 236,
 201,
 138,
 201,
 252,
 45,
 124,
 171,
 184,
 109,
 18,
 124,
 142,
 212,
 47,
 181,
 229,
 117,
 120,
 236,
 94,
 194,
 92,
 112,
 28,
 86,
 72,
 38,
 27,
 186,
 19,
 25,
 127,
 20,
 196,
 2,
 242,
 213,
 154,
 204,
 99,
 198,
 17,
 197,
 70,
 195,
 195,
 163,
 105,
 239,
 148,
 77,
 80,
 176,
 181,
 177,
 113,
 248,
 12,
 95,
 77,
 238,
 83,
 0,
 18,
 237,
 88,
 154,
 195,
 145,
 253,
 174,
 26,
 174,
 2,
 121,
 119,
 70,
 41,
 76,
 190,
 245,
 158,
 7,
 170,
 47,
 215,
 3,
 126,
 85,
 99,
 4,
 75,
 13,
 27,
 164,
 102,
 39,
 59,
 41,
 250,
 138,
 194,
 147,
 243,
 136,
 83,
 2,
 173,
 143,
 178,
 213,
 203,
 47,
 201,
 163,
 49,
 187,
 51,
 63,
 217,
 217,
 167,
 161,
 40,
 180,
 1,
 143,
 79,
 53,
 191,
 20,
 137,
 222,
 11,
 171,
 251,
 6,
 150,
 40,
 209,
 250,


### How to get the ember's features

In [14]:
def get_ember_feature(data):
    ember_f = np.zeros((len(data.mw_em_f), 2351), dtype=float)
    for index, item in data.iterrows():
        ember_f[index, :] = item['mw_em_f'].split(';')
    return ember_f

ember_data = get_ember_feature(x_train)

In [15]:
ember_data[:5]

array([[ 2.58349180e-01,  9.17421561e-03,  5.90146799e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.09979928e-01,  9.29322001e-03,  3.51542723e-03, ...,
         0.00000000e+00,  0.00000000e+00, -1.00000000e+00],
       [ 1.12707168e-01,  9.48794466e-03,  1.89540479e-02, ...,
         0.00000000e+00,  3.00000000e+00, -3.00000000e+00],
       [ 8.66679430e-01,  2.42161099e-03,  1.41865504e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.01829666e-01,  7.23230932e-03,  5.99512970e-03, ...,
         1.00000000e+00, -3.00000000e+00,  2.00000000e+00]])